17.31 Minimizing tax liability

In [1]:
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt

In [10]:
C = 2300
n = 10
b = np.array([400,80,400,200,400,400,80,400,100,500])
v = np.array([500,100,500,200,700,300,120,300,150,600])
L = 4
rho_l = 0.2
rho_s = 0.3

In [6]:
# check if it is possible that the net gain is negative
s = cp.Variable(n, nonneg=True)
objective = cp.Minimize(0)
constraints = [
    s <= v,
    cp.sum(s) == C,
    cp.sum(cp.multiply(s, (v - b) / v)) <= 0
]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print(s.value)

None


In [27]:
# assume long term loss
s = cp.Variable(n, nonneg=True)
objective = cp.Minimize(rho_s * (cp.sum(cp.multiply(s, (v - b) / v)[L:]) - cp.sum(cp.multiply(s, (v - b) / v)[:L])))
constraints = [
    s <= v,
    cp.sum(s) == C,
    cp.sum(cp.multiply(s, (v - b) / v)[:L]) <= 0
]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print(objective.value)

78.00000002045641


In [28]:
# assume short term loss
s = cp.Variable(n, nonneg=True)
objective = cp.Minimize(rho_l * (cp.sum(cp.multiply(s, (v - b) / v)[:L]) - cp.sum(cp.multiply(s, (v - b) / v)[L:])))
constraints = [
    s <= v,
    cp.sum(s) == C,
    cp.sum(cp.multiply(s, (v - b) / v)[L:]) <= 0
]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print(objective.value)
s_opt = s.value
min_tax = objective.value

24.266666667008604


In [26]:
# assume no short / long term loss
s = cp.Variable(n, nonneg=True)
objective = cp.Minimize(rho_l * (cp.sum(cp.multiply(s, (v - b) / v)[:L])) + rho_s * cp.sum(cp.multiply(s, (v - b) / v)[L:]))
constraints = [
    s <= v,
    cp.sum(s) == C,
    cp.sum(cp.multiply(s, (v - b) / v)[L:]) >= 0,
    cp.sum(cp.multiply(s, (v - b) / v)[:L]) >= 0,
]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print(objective.value)

24.266666684548063


In [29]:
s_proportion = C / np.sum(v) * v
N_l_proportion = np.sum(np.multiply(s_proportion, (v - b) / v)[:L])
N_s_proportion = np.sum(np.multiply(s_proportion, (v - b) / v)[L:])
print(N_l_proportion>0, N_s_proportion>0)

True True


In [30]:
print('optimal sell:', s_opt)
print('minimum tax liability:', min_tax)
print('tax liability for proportional sell:', N_l_proportion * rho_l + N_s_proportion * rho_s)

optimal sell: [269.84057316  66.98552035 269.84057316 200.          23.33333334
 300.         119.99999999 300.         150.         600.        ]
minimum tax liability: 24.266666667008604
tax liability for proportional sell: 86.82997118155619
